In [210]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np


Родительский класс, который получает запрос на сбор данных с сайта (библиотека requests)
    и собирает все данные с сайта (библиотека BeautifulSoup)

In [178]:
class Parser:

    """Родительский класс, который получает запрос на сбор данных с сайта (библиотека requests)
    и собирает все данные с сайта (библиотека BeautifulSoup)"""

    def __init__(self, url_template):
        self.url_template = url_template
        if requests.get(self.url_template).status_code == 200:#получая положительный ответ от сервера,начинаем собирать данные
            self.__collecting_information()
        else:
            print("Невозможно получить информацию с данного сайта!")

    def __collecting_information(self):
        soup = bs(requests.get(self.url_template).text, "html.parser")# парсим всю страницу
        return soup

Дочерний класс, который выдает всю запрашиваемую пользователем информацию по  валютам

In [179]:
class ParserCurrency (Parser):

    """Дочерний класс, который выдает всю запрашиваемую пользователем информацию по  валютам"""

    def __init__(self, url_template, tag, name_class, headers=None, currencies=None): 
        super().__init__(url_template)
        self.__tag = tag
        self.__name_class = name_class
        self.__headers = headers
        self.__currencies = currencies # пользователь не должен иметь доступ к фрейму, полученные данные нельзя менять
        if self.__headers == None:
            self.__headers = []
        if self.__currencies == None:
            self.__currencies = []
        self.__information_processing()

    def __information_processing(self): #достаем нужную информацию (таблица валют)
        information = self._Parser__collecting_information().find_all(self.__tag, class_=self.__name_class)
        for name in information:
            a = name.tbody

        for i in a.find_all("th"):#из тега <th> достаем заголовки из таблицы
            title = i.text
            self.__headers.append(title)

        for i in a.find_all("tr")[1:]: 
            info = i.find_all("td") #из тега <td> достаем данные, ледащие в  таблице
            self.__currencies.append([j.text for j in info])
    def return_currencies(self):
        return self.__currencies
    
    def return_headers(self):
        return self.__headers
    

In [180]:
parser1 = ParserCurrency("https://www.cbr.ru/currency_base/daily/", "table", "data")


In [181]:
class Table:
    
    """Класс, который выводит таблицу с указанными данными (библиотека pandas)"""
    
    def __init__(self, data, columns, name_csv):
        self.data=data
        self.columns=columns
        self.name_csv=name_csv
        self.frame = pd.DataFrame(data=self.data, index=(range(1, len(self.data)+1)), #скрыть frame
                             columns=self.columns)
        self.save_csv()
    
    def save_csv(self): #сохраняем csv файл
        self.frame.to_csv(self.name_csv, index=False)

    def reading_csv(self):# читаем из csv файла 
        return pd.read_csv(self.name_csv)


In [182]:
class Table_currencies(Table):
    
    
    def get_table(self):
        
        self.frame['Курс'] = self.frame['Курс'].apply(lambda x: float(x.replace(',', '.')))
        self.frame['Единиц'] = self.frame['Единиц'].apply(lambda x: int(x))
        self.frame.loc[35]=['643', 'RUS', 1, 'Российский рубль', 1]
        self.frame['Курс']=self.frame['Курс']/self.frame['Единиц']
        self.frame.index=self.frame['Букв. код']
        self.frame=self.frame.drop(['Цифр. код', 'Букв. код', 'Единиц'], axis=1)
        return self.frame
        

In [183]:
table = Table_currencies(parser1.return_currencies(),parser1.return_headers(), "Currencies.csv")
table.get_table()

,Валюта,Курс
Букв. код,,
AUD,Австралийский доллар,58.267900
AZN,Азербайджанский манат,46.487500
AMD,Армянских драмов,0.167850
BYN,Белорусский рубль,28.257800
BGN,Болгарский лев,43.953700
BRL,Бразильский реал,16.907100
HUF,Венгерских форинтов,0.228916
KRW,Вон Республики Корея,0.063893
HKD,Гонконгский доллар,10.098200


In [221]:
class Currency: 
    
    """В этом классе храним информацию о валютах"""
    
    __data = table.frame
    
    def print_data():
        return Currency.__data
    
Currency.print_data()

,Валюта,Курс
Букв. код,,
AUD,Австралийский доллар,58.267900
AZN,Азербайджанский манат,46.487500
AMD,Армянских драмов,0.167850
BYN,Белорусский рубль,28.257800
BGN,Болгарский лев,43.953700
BRL,Бразильский реал,16.907100
HUF,Венгерских форинтов,0.228916
KRW,Вон Республики Корея,0.063893
HKD,Гонконгский доллар,10.098200


In [225]:
class USER:
    
    """класс пользователя"""
    
    def __init__(self, e_wallet=pd.Series(0.0, index=Currency.print_data().index), user_information={'surname':'Rudneva', 'name':'Irina', 'age':19}):
        self.__e_wallet = e_wallet 
        self.__e_wallet['RUS']=100000.0 #хранит series user'а ( с ключами цифр.код, 
        #а в значениях количество данной валюты в кошельке пользователя ) доступ для пользователя убираем, т.к.он не может себе 
        #добавлять сам денег, только проводя операцию обмена!
        self.user_information = user_information
        
    def information_money(self): #смотрим, что лежит в кошельке
        return self.__e_wallet[self.__e_wallet>0]
        
        
user1=USER()
user1.information_money() 
#user1._USER__e_wallet

Букв. код
RUS    100000.0
dtype: float64

In [226]:
class Bank():
    
    """класс банка, который хранит всю информацию по валютам, умеет делать опреацию обмена"""
    
    def __init__(self, instruments=Currency.print_data(), money_resource=user1._USER__e_wallet):
        self.__instruments = instruments
        self.__money_resource = money_resource
        
    def exchange_currencies(self, count, swap, purchase): 
    #функция обмена валют (в аргументах кол-во покупаемой валюты; валюта, которую меняем; приобретаемая валюта) 
        if self.__money_resource[swap]>=(self.__instruments['Курс'][purchase]*float(count)/self.__instruments['Курс'][swap]):
            self.__money_resource[swap]-=(self.__instruments['Курс'][purchase]*float(count)/self.__instruments['Курс'][swap])
            self.__money_resource[purchase]+=count
        else:
            print('Недостаточно средств')
            
transaction= Bank()
transaction.exchange_currencies(1, 'RUS', 'USD')
transaction.exchange_currencies(79.0287 , 'USD', 'RUS')
transaction.exchange_currencies(100, 'RUS', 'KZT')

user1.information_money()

Букв. код
KZT      100.0000
RUS    99982.4322
dtype: float64

In [224]:
class Finance_instrument:
    
    """Базовый класс, содержащие основные критерии финансовых инструментов """
    
    def __init__(self, time, instrument):
        self.time=time
        self.instrument=instrument
       
        
        

In [11]:
class futures(Finance_instrument):
    def __init __(self, time, instrument):
        super().__init__(time, instrument)

SyntaxError: invalid syntax (<ipython-input-11-2cc32d3781a7>, line 2)

SyntaxError: invalid syntax (<ipython-input-58-014dce922c30>, line 3)

In [ ]:
#преобразовываем список списков в удобный словарь, оставляем нужные данные
import copy 
CURRENCY = copy.deepcopy(parser1.print_currencies()) 
HEADERS = copy.deepcopy(parser1.print_headers())
keys=[]
CURRENCY1 = []
for currency in CURRENCY:
    keys.append(currency[0])
    currency.pop(0)
    currency.pop(0)
    currency[0]=int(currency[0])
    
    currency[2]=currency[2].replace(',', '.')
    currency[2]=float(currency[2])
    CURRENCY1.append(dict(zip(HEADERS[2:], currency)))   
dic_currencies = dict(zip(keys, CURRENCY1))

class Currency: 
    
    """В этом классе храним словарь с валютами"""
    
    __data = dic_currencies
    __data['643']={'Единиц': 1, 'Валюта': 'Рубль', 'Курс': 1.0} #добавляем для удобства
    
    def print_data():
        return Currency.__data
    
Currency.print_data()

In [ ]:
class Table:
    
    """Класс, который выводит таблицу с указанными данными (библиотека pandas)"""
    def __init__(self, name_csv):
        self.name_csv=name_csv
#     def __init__(self, data, columns, name_csv):
#         self.__data = data
#         self.__columns = columns #и снова скрываем доступ для пользователя
#         self.name_csv = name_csv
#         print(self.filling_table())
#         self.save_csv()

#     def filling_table(self): # делаем табличку, из имеющихся данных
#         frame = pd.DataFrame(data=self.__data, index=(range(1, len(self.__data)+1)), columns=self.__columns)
#         return frame
    def filling_table(self): # делаем табличку, из имеющихся данных
        frame = pd.DataFrame(data=parser1.print_currencies(), index=(range(1, len(parser1.print_currencies())+1)), 
                             columns=parser1.print_headers())
        return frame

#     def save_csv(self): #сохраняем csv файл
#         self.filling_table().to_csv(self.name_csv, index=False)
        
    def save_csv(self): #сохраняем csv файл
        self.filling_table().to_csv(self.name_csv, index=False)

    def reading_csv(self):# читаем из csv файла 
        return pd.read_csv(self.name_csv)


In [ ]:
# class Instrument:
#     instruments=[]
#     def __init__ (self, name, value=0):
#         self.name=name
#         self.value=value
#         Instrument.instruments.append([self.name, self.value])
        
# class Currency(Instrument):
#     instruments=[]
#     def __init__(self, name, value=0):
#         super().__init__(name, value)
#         Currency.instruments.append([self.name, self.value])
        
# rub=Currency("Рубли", 100000)
# usd=Currency("Доллары", 100)
# euro=Currency("Евро")

# class Metal(Instrument):
#     instruments=[]
#     def __init__(self, name, value=0):
#         super().__init__(name, value)
#         Metal.instruments.append([self.name, self.value])

# gold=Metal("Золото")
# silver=Metal("Серебро")